In [1]:
import json
import re

In [22]:
data=[]
# for line in open('C:/Users/MoonJu/Documents/분석 프로젝트/Document summary/extractive_test_v2.jsonl', 'r', encoding='utf-8'):
#     data.append(json.loads(line))

# for line in open('C:/Users/Playdata/Downloads/extractive_test_v2.jsonl', 'r', encoding='utf-8'):
#     data.append(json.loads(line))

for line in open('D:/BigData/document_extraction_data/extractive_test_v2.jsonl', 'r', encoding='utf-8'):
    data.append(json.loads(line))

# for line in open('/content/gdrive/My Drive/빅데이터/문서 추출요약/extractive_test_v2.jsonl', 'r', encoding='utf-8'):
#     data.append(json.loads(line))

In [23]:
# 전처리
def CleanText(readData, Num=False, Eng=False):
# 번역 추가 해볼것
# 이메일
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ ]+', '', readData)

    return text.strip(' ')

In [24]:
# 사용할 데이터 정리
clean_article = []
ori_article = []
id_list = []
for i in range(len(data)):
    s1 = []
    s2 = []
    id_list.append(data[i]['id'])
    for j in range(len(data[i]['article_original'])):
        s1.append(data[i]['article_original'][j])
        s2.append(CleanText(data[i]['article_original'][j]))
    ori_article.append(s1)
    clean_article.append(s2)

In [25]:
clean_article[0]

['석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 을 쌓아두고 시위를 벌이고 있다',
 '석문간척지 임차법인협의회이하 간척지협의회가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 일 한국농어촌공사 당진지사 앞에 공공비축벼 을 쌓고 시위를 벌였다',
 '개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 당 원으로 책정된 임대료를 현재 쌀 판매가격인 원대로 인하할 것을 요구하고 있다',
 '이들은 지난 월 일 농림축산식품부에 탄원서를 제출했지만 임대료 인하는 올해 이후에나 가능하다고 통보받은 상황이다',
 '게다가 임차법인들의 계약기간이 올해 만료되기 때문에 임대료를 인하해도 지난 년 동안의 손실 보상은 받을 수 없는 상황이다',
 '이에 간척지협의회는 계약기간을 년 연장하고 연장된 기간 동안 인하된 임대료를 적용해 지난 년 간의 손실에 대해 보상할 것을 제안했다',
 '더불어 요구사항이 받아들여지지 않을 경우 벼 야적시위를 시작한 날짜인 지난해 월 일자로 임대료를 벼로 납부하겠다는 입장이다',
 '김재용 봉치영농조합법인 조합원은 현재 한국농어촌공사의 답변을 기다리고 있다며 상황을 지켜본 뒤 추가적인 야적 시위 여부를 결정할 계획이라고 말했다',
 '이어 법을 만들고 집행할 때 현실성 있게 만들어야 한다며 농민이 정부를 믿을 수 있도록 조속히 해결책을 마련해주길 바란다고 덧붙였다']

In [45]:
from typing import List
import numpy as np
from sklearn.preprocessing import normalize
from textrank import KeysentenceSummarizer
from konlpy.tag import Komoran, Kkma, Okt, Mecab

kkma = Kkma()
komoran = Komoran()
okt = Okt()
# mecab = Mecab()

def komoran_tokenizer(sent):
    words = kkma.pos(sent, join=True) #textrank 에서는 명사, 동사, 형용사만 사용 +어간
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)] # komoran
#     words = [w for w in words if ('/NP' in w or '/NN' in w or '/XR' in w or '/VA' in w or '/VV')] # kkma
    return words

summarizer = KeysentenceSummarizer(
#     tokenize = lambda x:x.split(),
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
#     similarity = 'cosine',
    window=2,
    verbose = True
)

idx_list = []
for sents in clean_article:
#     print(sents)
    keysents = summarizer.summarize(sents, topk=3)
    idx = []
    for sent_idx, rank, sent in keysents:
        idx.append(sent_idx)
    idx_list.append(idx)

TypeError: __init__() got an unexpected keyword argument 'window'

In [37]:
result = []
for i in range(len(idx_list)) :
    ori_str = []
    for j in range(3) :
        ori_str.append(ori_article[i][idx_list[i][j]])
    result.append('\n'.join(ori_str))

In [38]:
len(idx_list)

9987

In [39]:
import pandas as pd
result = pd.DataFrame(
                {'id' : id_list,
                 'summary' : result},
                )

In [42]:
result

,id,summary
0,500733466,43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당...
1,500742482,벌떼해장국의 주 메뉴는 뼈해장국과 감자탕이다.\n또한 벌떼해장국의 추천 메뉴는 콩나...
2,500742484,"반면 면천면 휘발유 평균 가격은 1478원으로 가장 낮았으며, 경유는 정미면이 평균..."
3,504213810,이에 대해 어기구 의원은 “제12차 장기 천연가스수급계획에는 제5기지를 2020년부...
4,505279620,정책자문위원으로 활동하고 있는 A씨는 “연 1회 전체회의를 진행하는데 회의시간이 짧...
...,...,...
9982,745338220,당초 시는 도시재생 중구 내항 1부두 일대가 국가시범지구로 선정되면 국비 250억 ...
9983,745367988,청와대 핵심 관계자는 이날 기자들과 만나 선거사범 267명의 복권이 이뤄진 것과 관...
9984,745368130,광주 감소율은 지난해 9월 -18.2%에 이어 14개월 만에 가장 큰 폭을 나타냈다...
9985,745368136,"향후 활용 방안을 모색 중인 헌책방은 다행히 앞으로도 책과 커피, 강연, 문화행사 ..."


In [41]:
# 결과 저장
# result.to_csv('C:/Users/MoonJu/Documents/분석 프로젝트/Document summary/extractive_submission_7.csv', encoding="utf-8", index=False)
result.to_csv('D:/BigData/document_extraction_data/extractive_submission_8.csv', encoding="utf-8", index=False)

In [ ]:
# 6 okt 잘못 됨
# similerity 변경
# 형태소 분석기 kkma 사용
# min_sim 변경
# 결과 summary 이상 원본 데이터 안들어감

# 전처리에서 번역 한번 시도해 볼것